In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import dmc
from process import processed_data

Tensorflow not installed


In [3]:
data = processed_data()
train_ids, test_ids = dmc.loading.load_ids('rawMirrored')
train, test = dmc.preprocessing.split_train_test(data, train_ids, test_ids)

## Full ensemble to classify and rerun on different data after learning

In [4]:
Forest, Network, Bayes, SVM = dmc.classifiers.Forest, dmc.classifiers.TheanoNeuralNetwork, dmc.classifiers.NaiveBayes, dmc.classifiers.SVM
Tree = dmc.classifiers.DecisionTree
raw_scaler, scaler, no_scaler = dmc.transformation.scale_raw_features, dmc.transformation.scale_features, None
no_sample = None
params = {
    'uuuu': {'sample': 20000, 'scaler': None, 'ignore_features': None, 'classifier': Forest},
    'uuuk': {'sample': 20000, 'scaler': None, 'ignore_features': None, 'classifier': Forest},
    'uuku': {'sample': 20000, 'scaler': None, 'ignore_features': None, 'classifier': Forest},
    'uukk': {'sample': 20000, 'scaler': None, 'ignore_features': None, 'classifier': Forest}, # Theano
    'ukuu': {'sample': 20000, 'scaler': None, 'ignore_features': None, 'classifier': Forest},
    'ukuk': {'sample': 20000, 'scaler': None, 'ignore_features': None, 'classifier': Forest},
    'ukku': {'sample': 20000, 'scaler': None, 'ignore_features': None, 'classifier': Forest},
    'ukkk': {'sample': 200000, 'scaler': None, 'ignore_features': None, 'classifier': Forest},
    'kuuu': {'sample': 20000, 'scaler': None, 'ignore_features': None, 'classifier': Forest},
    'kuuk': {'sample': 20000, 'scaler': None, 'ignore_features': None, 'classifier': Forest}, # Theano
    'kuku': {'sample': 20000, 'scaler': None, 'ignore_features': None, 'classifier': Forest}, 
    'kukk': {'sample': 200000, 'scaler': None, 'ignore_features': None, 'classifier': Forest}, # Theano
    'kkuu': {'sample': 200000, 'scaler': None, 'ignore_features': None, 'classifier': Forest},
    'kkuk': {'sample': 200000, 'scaler': None, 'ignore_features': None, 'classifier': Forest},
    'kkku': {'sample': 20000, 'scaler': None, 'ignore_features': None, 'classifier': Forest},
    'kkkk': {'sample': 200000, 'scaler': None, 'ignore_features': None, 'classifier': Forest}
}
"""
for e in ensemble.splits:
    ensemble.splits[e] = {**ensemble.splits[e], **params[e]}
print(ensemble.splits)"""

'\nfor e in ensemble.splits:\n    ensemble.splits[e] = {**ensemble.splits[e], **params[e]}\nprint(ensemble.splits)'

In [5]:
# allocate memory
data = train_ids = test_ids = None
####
ensemble = dmc.ensemble.ECEnsemble(train, test, params)

In [6]:
ensemble.transform()

In [7]:
ensemble.classify()

uuuk precision 0.6738712776176753 size 2082
uuku precision 0.8571428571428572 size 14
uukk precision 0.6620036101083033 size 22160
ukuk precision 0.6591540598950294 size 3239
ukku precision 0.6666666666666667 size 12
ukkk precision 0.6458202842688509 size 62265
kuuk precision 0.6667462401527811 size 4189
kukk precision 0.6615628792829802 size 42844
kkuk precision 0.645628078817734 size 6496
kkkk precision 0.6456538300589074 size 93197
OVERALL: 0.650931508935


In [ ]:
for split in ensemble.splits:
    print(len(ensemble.splits[split][1]), split)

Print splits and names to know where to not drop customer for naive bayes.

### Baseline
```
known_articleID-unknown_customerID-unknown_voucherID-known_productGroup : size  4189  prec  0.659823346860826
--------------------------------------
known_articleID-known_customerID-known_voucherID-known_productGroup : size  93197  prec  0.6745603399251049
--------------------------------------
unknown_articleID-known_customerID-known_voucherID-known_productGroup : size  62265  prec  0.6728338552959126
--------------------------------------
unknown_articleID-unknown_customerID-known_voucherID-unknown_productGroup : size  14  prec  0.8571428571428572
--------------------------------------
known_articleID-known_customerID-unknown_voucherID-known_productGroup : size  6496  prec  0.6657943349753694
--------------------------------------
unknown_articleID-unknown_customerID-known_voucherID-known_productGroup : size  22160  prec  0.6546931407942238
--------------------------------------
unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup : size  2082  prec  0.6652257444764649
--------------------------------------
unknown_articleID-known_customerID-known_voucherID-unknown_productGroup : size  12  prec  0.6666666666666667
--------------------------------------
known_articleID-unknown_customerID-known_voucherID-known_productGroup : size  42844  prec  0.6607459620950424
--------------------------------------
unknown_articleID-known_customerID-unknown_voucherID-known_productGroup : size  3239  prec  0.6687249150972523
--------------------------------------
Overall : 0.6690881106816972 78260
```


### SVM with 50k sample
```

unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6714697406340058 2082
unknown_articleID-unknown_customerID-known_voucherID-unknown_productGroup 0.8571428571428572 14
unknown_articleID-unknown_customerID-known_voucherID-known_productGroup 0.6607851985559567 22160
unknown_articleID-known_customerID-unknown_voucherID-known_productGroup 0.6644025933930225 3239
unknown_articleID-known_customerID-known_voucherID-unknown_productGroup 0.6666666666666667 12
unknown_articleID-known_customerID-known_voucherID-known_productGroup 0.6448727214325864 62265
known_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6719980902363333 4189
known_articleID-unknown_customerID-known_voucherID-known_productGroup 0.6569881430305293 42844
known_articleID-known_customerID-unknown_voucherID-known_productGroup 0.6366995073891626 6496
known_articleID-known_customerID-known_voucherID-known_productGroup 0.6434649183986609 93197
```

In [ ]:
def shuffle(df: pd.DataFrame) -> pd.DataFrame:
    return df.reindex(np.random.permutation(df.index))

In [ ]:
for split in ensemble.splits:
    ensemble.splits[split] = (shuffle(ensemble.splits[split][0])[:200000], ensemble.splits[split][1])

In [ ]:
len(ensemble.test)

In [ ]:
splits = len(ensemble.splits)
scalers = [dmc.transformation.scale_raw_features] * splits
ignore = [None] * splits
ensemble.transform(scalers=scalers, ignore_features=ignore)

In [ ]:
res = []
for k in ensemble.splits:
    if len(ensemble.splits[k]['test'][1]) < 3000:
        continue
    Xt, yt = ensemble.splits[k]['train'][0], ensemble.splits[k]['train'][1]
    Xe, ye = ensemble.splits[k]['test'][0], ensemble.splits[k]['test'][1]
    fts, clf = ensemble.splits[k]['features'], dmc.classifiers.TheanoNeuralNetwork
    clf = clf(Xt, yt)
    prec = dmc.evaluation.precision(ye, clf(Xe))
    res.append((k, prec, len(ye)))
    print(k, prec, len(ye), clf)

In [ ]:
resF = []
for k in ensemble.splits:
    if len(ensemble.splits[k]['test'][1]) < 3000:
        continue
    Xt, yt = ensemble.splits[k]['train'][0], ensemble.splits[k]['train'][1]
    Xe, ye = ensemble.splits[k]['test'][0], ensemble.splits[k]['test'][1]
    fts, clf = ensemble.splits[k]['features'], dmc.classifiers.Forest
    clf = clf(Xt, yt)
    prec = dmc.evaluation.precision(ye, clf(Xe))
    resF.append((k, prec, len(ye)))
    print(k, prec, len(ye), clf)

In [ ]:
for k in ensemble.splits:
    Xt, yt = ensemble.splits[k]['train'][0], ensemble.splits[k]['train'][1]
    Xe, ye = ensemble.splits[k]['test'][0], ensemble.splits[k]['test'][1]
    fts, clf = ensemble.splits[k]['features'], dmc.classifiers.NaiveBayes
    clf = clf(Xt, yt)
    prec = dmc.evaluation.precision(ye, clf(Xe))
    print(k, prec, len(ye), clf)

In [ ]:
Forest, Network, Bayes, SVM = dmc.classifiers.Forest, dmc.classifiers.TheanoNeuralNetwork, dmc.classifiers.Bayes, dmc.classifiers.SVM
raw_scaler, no_scaler = dmc.transformation.scale_raw_features, None
no_sample = None
params = {
    'uuuu': 
}